# <u>Solution :

# 1. Importing the Dataset and doing Data Preprocessing on the dataset

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns
import datetime

from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [ ]:
# reading the dataset
# Note: Our Dataset is very large so setting low_memory to false
dataset = pd.read_csv('Service_Requests_from_2010_to_Present.csv', low_memory = False)

In [ ]:
# View the top 5 elements of the dataset
dataset.head(5)

In [ ]:
# shape of the dataset
dataset.shape

In [ ]:
dataset.info()

#### There are a lot of columns in our dataset but I don't need all of them. So I will drop the columns that have a very large number of null values in it. Also I don't need the unnecessary columns, so I will drop them.

In [ ]:
# viewing the columns
dataset.columns

- Columns with most of its values as null are dropped
- Agency and Agency Name are more or less same, so I am dropping Agency Name
- Dropping other unnecessary columns like Location, Incident Adrress, Street Name, X Coordinate (State Plane), Y Coordinate (State Plane)  as we are already considering LATITUDE and LONGITUDE
- Dropping other unnecessary columns

In [ ]:
# dropping the irrelevant columns
drop_columns = ['Agency Name','Incident Address','Street Name','Cross Street 1','Cross Street 2','Intersection Street 1',
'Intersection Street 2','Address Type','Park Facility Name','Park Borough','School Name',
'School Number','School Region','School Code','School Phone Number','School Address','School City',
'School State','School Zip','School Not Found','School or Citywide Complaint','Vehicle Type',
'Taxi Company Borough','Taxi Pick Up Location','Bridge Highway Name','Bridge Highway Direction',
'Road Ramp','Bridge Highway Segment','Garage Lot Name','Ferry Direction','Ferry Terminal Name','Landmark',
'X Coordinate (State Plane)','Y Coordinate (State Plane)','Due Date','Resolution Action Updated Date','Community Board','Facility Type',
'Location']

dataset = dataset.drop(drop_columns, axis=1)

In [ ]:
# checking the shape of our dataset
dataset.shape

#### So we are upto 14 columns from 53...That's a lot of reduction !

- ### Now check for null values, data type of the columns etc.

In [ ]:
# checking the info of the dataset
dataset.info()

In [ ]:
# checking the number of null values in the columns
dataset.isnull().sum()

In [ ]:
# choosing the closed cases only to eliminate the null values

dataset = dataset[dataset['Status'] == 'Closed']

In [ ]:
dataset.isnull().sum()

In [ ]:
# since all the cases are of closed cases we can now drop the column Status as every value of its data are same
dataset = dataset.drop(['Status'], axis = 1)

In [ ]:
dataset.shape

In [ ]:
# out of the remaining columns only Descriptor, Latitude and Longitude has over 1k null values
dataset = dataset[(dataset['Descriptor'].notnull()) & (dataset['Latitude'].notnull()) &(dataset['Longitude'].notnull())]

dataset.info()

In [ ]:
# rechecking remaining null values
dataset.isnull().sum()

In [ ]:
# still there are few null values in our dataset. we will remove them
dataset = dataset[(dataset['Location Type'].notnull()) & (dataset['Incident Zip'].notnull()) &(dataset['City'].notnull())]

# rechecking for null values
dataset.isnull().sum()

-- So all Null values are removed from our dataset

In [ ]:
# checking the shape our datset
dataset.shape

**2. Read or convert the columns ‘Created Date’ and Closed Date’ to datetime datatype and create a new column ‘Request_Closing_Time’ as the time elapsed between request creation and request closing. (Hint: Explore the package/module datetime)**

In [ ]:
# converting 'Created Date' and 'Closed Date' to datetime datatype

cols = ['Created Date', 'Closed Date']
for col in cols:
    dataset[col] = pd.to_datetime(dataset[col],infer_datetime_format=True)
    
# creating a new column Request_Closing_Time
dataset['Request_Closing_Time'] = dataset[cols[1]] - dataset[cols[0]]

# viewing the info to see the data types
dataset.info()

**3. Provide major insights/patterns that you can offer in a visual format (graphs or tables); at least 4 major conclusions that you can come up with after generic data mining.**

In [ ]:
# applying describe on the dataset
dataset.describe()

In [ ]:
# viewing the columns
dataset.columns

### We will analyse Agency, Complaint Type, Descriptor, Location Type, City, Borough 

In [ ]:
# All the complaints are under the same agency
dataset['Agency'].value_counts()

In [ ]:
# complaint types
dataset['Complaint Type'].value_counts()

In [ ]:
# plotting the complaint types
dataset['Complaint Type'].value_counts().plot(kind = 'bar', figsize=(15, 7), title='Complaint Types', ylabel='Count', grid=True)

- so <b>Blocked Driveway</b> is the Maximum Complaint type followed by Illegal Parking, Noise-Street/Sidewalk, Noise-Commercial

In [ ]:
# Descriptors
dataset['Descriptor'].value_counts()

In [ ]:
# plotting top 10 Descriptors
dataset['Descriptor'].value_counts().head(10).plot(kind='barh', grid=True, figsize=(10,5), title='Top 10 Descriptors')

- So <b>Loud Music/Party</b> is the <u>maximum</u> descriptor for the complaints followed by No Access, Posted Parking Sign Violation and Loud Taking.

In [ ]:
# Top 10 Location Type
dataset['Location Type'].value_counts().head(10)

In [ ]:
# plotting Top 10 Location Type
dataset['Location Type'].value_counts().head(10).plot(kind='barh', grid=True, figsize=(10, 5), title='Top 10 Location Type')

- So we see that the Location Type of <b>Street/Sidewalk</b> is a lot more than any other members of its category

In [ ]:
# City
dataset['City'].value_counts().head(10)

In [ ]:
# plotting the cities
dataset['City'].value_counts().head(10).plot(kind='barh', grid=True, figsize=(10, 5), title='City Column', ylabel='Cities')
plt.xlabel('Complaint Counts')

- So most complaints are from <b>BROOKLYN</b> followed New York, Bronx, Staten Island in City wise

In [ ]:
# Borough
dataset['Borough'].value_counts().head(10).plot(kind='barh', grid=True, figsize=(10, 5), title='Borough Column', ylabel='Borough')
plt.xlabel('Complaint Counts')

### Till now we only analyse one column. Lets analyse Borough and Complaint Types

In [ ]:
# Top Complaints
top_6_complaints = dataset['Complaint Type'].value_counts()[:6].keys()
top_6_complaints

In [ ]:
# Borough per Complaints
borough_complaints = dataset.groupby(['Borough', 'Complaint Type']).size().unstack()
borough_complaints = borough_complaints[top_6_complaints]
borough_complaints

In [ ]:
# Plotting Borough per Complaint Type 
col_number = 2
row_number = 3
fig, axes = plt.subplots(row_number,col_number, figsize=(12,8))

for i, (label,col) in enumerate(borough_complaints.items()):
    ax = axes[int(i/col_number), i%col_number]
    col = col.sort_values(ascending=True)[:15]
    col.plot(kind='barh', ax=ax, grid=True)
    ax.set_title(label)
    
plt.tight_layout()

<u>ANALYSIS:</u>
    
    - Blocked Driveway is maximum in QUEENS
    - Illegal Parking is maximum in BROOKLYN
    - Noise - Street/Sidewalk is maximum in MANHATTAN
    - Noise - Commercial is maximum in MANHATTAN
    - Derelict Vehicle is maximum in QUEENS
    - Noise - Vehicle is maximum in QUEENS


- Similarly for Complaints per Borough

In [ ]:
# similarly for Complaints per Borough
top_borough = dataset['Borough'].value_counts().keys()

complaint_per_borough = dataset.groupby(['Complaint Type', 'Borough']).size().unstack()
complaint_per_borough = complaint_per_borough[top_borough]
complaint_per_borough

In [ ]:
# Plotting Complaints per Borough
col_number = 2
row_number = 3
fig, axes = plt.subplots(row_number,col_number, figsize=(12,10))

for i, (label,col) in enumerate(complaint_per_borough.items()):
    ax = axes[int(i/col_number), i%col_number]
    col = col.sort_values(ascending=True)[:15]
    col.plot(kind='barh', ax=ax, grid=True)
    ax.set_title(label)
    
plt.tight_layout()

<u>ANALYSIS:</u>

    - BROOKLYN, QUEENS and BRONX has most complaints of Blocked Driveway.
    - MANHATTAN has most complaints of Noise - Street/Sidewalk.
    - STATEN ISLAND has most complaints of Illegal Parking.

**4. Order the complaint types based on the average ‘Request_Closing_Time’, grouping them for different locations.**

In [ ]:
# Creating a column Request_Closing_Time_in_Hours for time in Hours
# and we will say the said complaint has been closed under x hours
dataset['Request_Closing_Time_in_Hours'] = dataset['Request_Closing_Time'].dt.total_seconds() / 3600 + 1

# viewing the Two columns side by side for first 20 entries
dataset[['Request_Closing_Time', 'Request_Closing_Time_in_Hours']].head(20)

In [ ]:
#  Ordering the complaint types based on the average ‘Request_Closing_Time’ in Hours, grouping them for different locations.
data_avg_time_in_hrs = dataset.groupby(['City', 'Complaint Type'])['Request_Closing_Time_in_Hours'].mean()
data_avg_time_in_hrs.head(10)

In [ ]:
# Creating a column Request_Closing_Time_in_Seconds for time in seconds
dataset['Request_Closing_Time_in_Seconds'] = dataset['Request_Closing_Time'].astype('timedelta64[s]')

# viewing the Three columns side by side
dataset[['Request_Closing_Time', 'Request_Closing_Time_in_Hours','Request_Closing_Time_in_Seconds']].head()

In [ ]:
#  Order the complaint types based on the average ‘Request_Closing_Time’ in seconds, grouping them for different locations.
data_avg_in_seconds = dataset.groupby(['City', 'Complaint Type']).Request_Closing_Time_in_Seconds.mean()
data_avg_in_seconds.head(10)

- <u>Other analysis works on Time

In [ ]:
# lets analyse the Request_Closing_Time
dataset['Request_Closing_Time'].describe()

### We will now analyse Complaint Types column on the basis of <u>Months</u> by refering to <u>Created Date</u>

In [ ]:
# for analysing on the basis of month we will need to separate months from Created Date column
dataset['Year-Month'] = dataset['Created Date'].apply(lambda x:datetime.datetime.strftime(x, '%Y-%m'))


In [ ]:
# viewing the months that we have in our dataset
dataset['Year-Month'].unique()

- Looks like we have incident complaints from <b>March</b> to <b>December</b>

In [ ]:
# plotting the month column
#dataset['Year-Month'].value_counts().plot()
monthly_incidents =  dataset.groupby('Year-Month').size().plot(figsize=(12,5),
                                                               title='Incident Counts on a monthly basis', ylabel='Counts')

- Are <b>January</b> and <b>Ferbruary</b> does not have any Complaints? Looks like nobody wants to go out in cold weather....😉
- We don't have any complaints from <b>January</b> and <b>Ferbruary</b> in our dataset because we might have eliminated them as <b>Null Values</b> earlier.

In [ ]:
# plotting Borough on the basis of Year-Month
dataset.groupby(['Year-Month','Borough']).size().unstack().plot(figsize=(15,7))
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

- <b>BROOKLYN</b> raised most cases all over and most of them were raised in <b>MAY-JUNE</b> and <b>SEPTEMBER</b>.

In [ ]:
# Plotting Year-Month on the basis of Borough
dataset.groupby(['Borough', 'Year-Month']).size().unstack().plot(figsize=(15,7))
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

- <b>DECEMBER</b> has raised least complaints.

In [ ]:
# Processing time per Borough on a monthly basis
dataset.groupby(['Year-Month','Borough'])['Request_Closing_Time_in_Hours'].mean().unstack().plot(figsize=(15,7),
                                                                        title='Processing time per Borough on a monthly basis');

- <b>BRONX</b> has the maximum Processing time every month even though it has the least complaints.


# 5. Perform a statistical test for the following:
**Please note: For the below statements you need to state the Null and Alternate and then provide a statistical test to accept or reject the Null Hypothesis along with the corresponding ‘p-value’.**


## a. Whether the average response time across complaint types is similar or not (overall)
## b. Are the type of complaint or service requested and location related?

In [ ]:
# viewing the columns
dataset.columns

In [ ]:
# lets see the average response time in seconds for different complaint types
avg_response_time = dataset.groupby(['Complaint Type']).Request_Closing_Time_in_Seconds.mean().sort_values(ascending=True)
avg_response_time

- <b>Null Hypothesis</b> -->>  Average response time across complaint types is similar(overall).
- <b>Alternate Hypothesis</b> -->> Average response time across complaint types is not similar(overall).

-- We can say that for overall case our <b>NULL HYPOTHESIS</b> is rejected as the Average Response Time in Seconds for different complaint types is different

# Testing for our Hypothesis

- Below We conduct ANOVA (Analysis of Variance) test for top 5 type of complaints

- For a 95% of confidence interval we choose our alpha as 0.05 for 5%

- Alpha(0.05) is the critical p-value, if our calculated p-value is less than alpha, it will give us strong evidence to reject Null Hypothesis.
>
- <b>if p < alpha(0.05) <b>: Reject Null Hypothesis, Average response time for all the complaints type is not same.</b>

- <b>if p > alpha(0.05) : Fail to reject Null Hypothesis, Average response time for all the complaints type is same.</b>

In [ ]:
# to calculate the p-value log of time taken to close the complaint per complaint is taken
#data = {}
#for complaint in dataset['Complaint Type'].unique():
 #   data[complaint] = np.log(dataset[dataset['Complaint Type']==complaint]['Request_Closing_Time_in_Seconds'])

In [ ]:
data = {}

# Iterate over unique complaint types
for complaint in dataset['Complaint Type'].unique():
    # Filter dataset to rows where 'Complaint Type' matches 'complaint'
    filtered_data = dataset[dataset['Complaint Type'] == complaint]
    
    # Filter out rows with non-numeric or invalid values in 'Request_Closing_Time_in_Seconds'
    valid_data = filtered_data.dropna(subset=['Request_Closing_Time_in_Seconds'])
    
    # Apply np.log() only to valid numeric values
    if not valid_data.empty:
        valid_data['Request_Closing_Time_in_Seconds'] = valid_data['Request_Closing_Time_in_Seconds'].astype(float)
        data[complaint] = np.log(valid_data['Request_Closing_Time_in_Seconds'])


In [ ]:
data.keys()

In [ ]:
for complaint in data.keys():
    print(data[complaint].std())

In [ ]:
# importing f_oneway from scipy.stats library
from scipy.stats import f_oneway
# taking top 5 complaints
stat, p = f_oneway(data['Blocked Driveway'], data['Illegal Parking'], data['Noise - Street/Sidewalk'],
                   data['Derelict Vehicle'], data['Noise - Commercial'])
print('Statistics= %.3f, p = %.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('We have Different distributions (reject H0)')

- <b>As our p-value is quite low , hence it is being converted to 0.0</b>

- Since our p-value is lower than our critical p-value, we will conclude that we have enough evidence to reject our Null Hypothesis and that is:

- Average response time for all the complaints type is not same.

### For relation between Complaint Type and Location
we will use Crosstab and Chi-square Test

In [ ]:
# getting crosstab from pandas
city_type = pd.crosstab(dataset['City'], dataset['Complaint Type'])

In [ ]:
# viewing first 5 entries of city_type
city_type.head()

In [ ]:
# Now we will do Chi-square Test

In [ ]:
# contigency table
table = city_type 
# table -->> The contingency table. The table contains the observed frequencies (i.e. number of occurrences) in each category.
# stat -->> chi2 or Test Statistic
# p -->> The p-value of the Test
# dof -->> Degrees of Freedom
# expected -->> The expected frequencies, based on the marginal sums of the table.
stat, p, dof, expected = chi2_contingency(table)

In [ ]:
print('The Degrees of Freedom are : {}'. format(dof))
print('The P-Value of the Testing is {}: '.format(p))
print('Expected values : \n')
print(expected)

In [ ]:
# interpreting test statistics
prob = 0.95 # as aplha is considered 0.05
critical_value = chi2.ppf(prob, dof)
critical_value

In [ ]:
print('Probability = {}, Critical Value = {}, Test statistic = {}'.format(prob, critical_value, stat))
print()
if abs(stat) >= critical_value:
    print('Dependent (Reject H0 or Null Hypothesis)')
else:
    print('Independent (Failed to reject Null Hypothesis)')


# interpreting the P-Value
alpha = 1 - prob
print('Significance : %.3f, P-Value : %.2f'%(alpha, p))
if p <= alpha:
    print('Dependent (Reject H0 or Null Hypothesis)')
else:
    print('Independent (Failed to reject Null Hypothesis)')